In [11]:
import cv2
import numpy as np

In [48]:
import pytesseract
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os 
from langdetect import detect

In [49]:
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'



In [3]:
 pytesseract. get_tesseract_version()

LooseVersion ('4.0.0-beta.1')

In [4]:

def extract_text(path,name) :

    ''' 
    Extracts text from pdf and 
    returns a list of all the strings which are in french

    '''
    PDF_file = name
    # Store all the pages of the PDF in a variable 
    pages = convert_from_path(PDF_file, 200) 
  
    # Counter to store images of each page of PDF to image 
    image_counter = 1
    
    # Iterate through all the pages stored above 
    for page in pages: 
        
        page2img(page,image_counter,name)
        # Increment the counter to update filename 
        image_counter += 1

    fr_list = image_to_text(image_counter,name)

    return fr_list


In [5]:
def page2img(page,img_cnt,name):
    
    # Declaring filename for each page of PDF as JPG 
    filename = name+"_"+str(img_cnt)+".jpg"
      
    # Save the image of the page in system 
    page.save(filename, 'JPEG') 

In [6]:
def text_to_list(text):
    to_list = []
    to_list = text.split('\n\n')
    to_list = list(filter(lambda x: x != " ", to_list))
    return to_list


In [107]:
def is_french(stringx) :
    try :
        if detect(stringx.lower()) == 'fr' :
            return True
    except :
        return False


def image_to_text(image_counter,name) :   
    ''' 
    Part #2 - Recognizing text from the images using OCR 
    '''
    
    # Variable to get count of total number of pages 
    filelimit = image_counter-1
  
    fr_list = []
  
    # Iterate from 1 to total number of pages 
    for i in range(1, filelimit + 1): 
  
        # Set filename to recognize text from 
    
        filename = name+"_"+str(i)+".jpg"
          
        # Recognize the text as string in image using pytesserct
        config = "--psm 3"
        img_preprocessing(filename)
        text = str(pytesseract.image_to_string(Image.open(filename))) #,config=config, lang                  #= 'fra')) 
  
        text = text.replace('-\n', '')

        text_list = text_to_list(text)
        for line in text_list :
            if is_french(line) or line.startswith('Artic') or line.startswith('§')  :
                fr_list.append(line.replace('\n',' '))
    
    return fr_list


In [104]:
def img_preprocessing(img_name):
    #def get_string(img_path, method):
    img = cv2.imread(img_name)

    img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    # Convert to gray
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply dilation and erosion to remove some noise
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)
    img = cv2.erode(img, kernel, iterations=1)
    #  Apply threshold to get image with only black and white
    img = cv2.threshold(cv2.GaussianBlur(img, (7, 7), 0), 0, 255, cv2.THRESH_BINARY +       cv2.THRESH_OTSU)[1]
    #img = cv2.threshold(cv2.GaussianBlur(img, (5, 5), 0), 0, 255, cv2.THRESH_BINARY +  cv2.THRESH_OTSU)[1]  
    #img = cv2.threshold(cv2.medianBlur(img, 3), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]      
    cv2.imwrite(img_name, img)
    

In [108]:
# Path of the pdf 
PATH = "src/notebooks/"
pdf_name = 'test.pdf'

french_text = extract_text(PATH,pdf_name)


In [109]:
french_text

['§1',
 '§2',
 '§3',
 'CONVENTION COLLECTIVE DE TRAVAIL',
 "DU 13 NOVEMBRE 2019 CONCLUE AU SEIN DE LA COMMISSION PARITAIRE AUXILIAIRE POUR OUVRIERS INSTAURANT ET DETERMINANTLA PROCEDURE DE MISE EN CEUVRE ET LES CONDITIONS D'OCTROI D'UN REGIME DE CHOMAGE AVEC COMPLEMENT D’ENTREPRISE EN EXECUTION DE LA CCT n° 138 et n° 139 CONCLUE AU SEIN DU CONSEIL NATIONAL DU TRAVAIL",
 "CHAPITRE | - CHAMP D'APPLICATION",
 'Article 1',
 "La présente convention collective de travail s'applique aux employeurs et aux ouvriers ressortissants du champ d’application de la commission paritaire auxiliaire pour ouvriers. Par «ouvriers », on entend les ouvriers masculins et féminins.",
 'CHAPITRE Il - PORTEE DE LA',
 "CONVENTION Article 2 §1 La présente convention a aussi bien pour objet d'instaurer un régime d'indemnité",
 "complémentaire applicable a _ certains travailleurs Agés licenciés que d'en déterminer la procédure de mise en ceuvre et les conditions d'octroi.",
 '§2',
 "Elle est conclue en vertu de l'ar

In [125]:
#french_text

In [14]:
import cv2
import numpy as np


# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    # threshold the image, setting all foreground pixels to
    # 255 and all background pixels to 0
    return cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

#skew correction
def deskew(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_not(gray)
    thresh = cv2.threshold(gray, 0, 255,
        cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    coords = np.column_stack(np.where(thresh > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h),
        flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)    
    return rotated

#template matching
def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)

In [15]:
img = cv2.imread('118.pdf_9.jpg')

deskew = deskew(img)
gray = get_grayscale(deskew)
thresh = thresholding(gray)
#rnoise = remove_noise(gray)
#dilate = dilate(gray)
#erode = erode(gray)
#opening = opening(gray)
#canny = canny(gray)

In [90]:
#def get_string(img_path, method):
img = cv2.imread('test.pdf_3.jpg')

img = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)
# Convert to gray
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# Apply dilation and erosion to remove some noise
kernel = np.ones((1, 1), np.uint8)
img = cv2.dilate(img, kernel, iterations=1)
img = cv2.erode(img, kernel, iterations=1)
#  Apply threshold to get image with only black and white
img = cv2.threshold(cv2.GaussianBlur(img, (7, 7), 0), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
#img = cv2.threshold(cv2.GaussianBlur(img, (5, 5), 0), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]  
#img = cv2.threshold(cv2.medianBlur(img, 3), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]  
cv2.imwrite('testpreprocessed.jpg', img)



True

In [103]:
# Recognize the text as string in image using pytesserct 
config = "--psm 3"
text = str(((pytesseract.image_to_string(Image.open('testpreprocessed.jpg'),config=config, lang = 'fra'))))
  
#text = text.replace('-\n', '')
fr1_list = []
text_list = text_to_list(text)
for line in text_list :
    if is_french(line) or line.startswith('Artic') :
        fr1_list.append(line.replace('\n',' '))
    


In [105]:
fr1_list

["Pendant huit jours à dater de cette communication écrite, l'employeur tient à la disposition des travailleurs un registre où ceux-ci peuvent consigner leurs observations. Pendant ce même délai de huit jours, le travailleur ou son représentant peut également communiquer ses observations à l'inspecteur social chef de direction de la Direction générale Contrôle des Lois sociales du Service public fédéral Emploi, Travail et Concertation sociale, du lieu d'établissement de l'entreprise. Le nom du travailleur ne peut être ni communiqué ni divulgué.",
 "Passé ce délai de huit jours, l'employeur dépose l'acte d'adhésion au Greffe de la Direction générale Relations collectives de travail du Service public fédéral Emploi, Travail et Concertation sociale, accompagné du registre.",
 'Article 5',
 "En cas de litiges relatifs à l'application des articles 3 et 4 à l'exception de ceux portant sur le règlement de travail, la cause est portée par la partie la plus diligente devant la commission parita

In [86]:
table_image_contour = cv2.imread('118preprocessed.jpg', 0)
table_image = cv2.imread('118preprocessed.jpg')

#ret, thresh_value = cv2.threshold(table_image_contour, 180, 255, cv2.THRESH_BINARY_INV)
#kernel = np.ones((5,5),np.uint8)
#dilated_value = cv2.dilate(thresh_value,kernel,iterations = 1)
#contours, hierarchy = cv2.findContours(dilated_value, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
#for cnt in contours:
#    x, y, w, h = cv2.boundingRect(cnt)
#    # bounding the images
#    if y < 50:
#        table_image = cv2.rectangle(table_image, (x, y), (x + w, y + h), (0, 0, 255), 1)
#
#cv2.namedWindow('detecttable', cv2.WINDOW_NORMAL)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))


In [87]:
import numpy
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, np.array(table_image).shape[1]//150))
eroded_image = cv2.erode(binary_image, vertical_kernel, iterations=5)

NameError: name 'binary_image' is not defined

In [13]:
img = cv2.imread("page_1.jpg")
text = pytesseract.image_to_string(img)
print(text)

 

COLLECTIEVE ARBEIDSOVEREENKOMST VAN 13
NOVEMBER 2019 GESLOTEN IN HET
AANVULLEND PARITAIR COMITE VOOR DE
WERKLIEDEN BETREFFENDE HET SECTORAAL
AKKOORD 2019-2020 — KOOPKRACHT

 

Hoofdstuk | Toepassingsgebied

Artikel 1

Deze collectieve arbeidsovereenkomst is van
toepassing op de werkgevers en de arbeiders van
de ondernemingen die onder de bevoegcheid
vallen van het Aanvullend Paritair Comité voor de
Werklieden. Onder “arbeiders” wordt verstaan
de mannelijke en vrouwelijke arbeiders.
Hoofdstuk II — Algemene bepaling

Artikel 2

Deze collectieve arbeidsovereenkomst is gesloten
in het kader van het Koninklijk besluit van 19 april
2019 tot uitvoering van artikel 7§1 van de Wet
van 26 juli 1996 ter bevordering van de
werkgelegenheid en en tot preventieve vrijwaring
van het concurrentievermogen.

Hoofdstuk II] — verhoging van minimumuurloon
en van effectief uurloon

Artikel 3

Het minimumuurloon en het effectieve uurloon
worden met 1,1% verhoogd vanaf 1/11/2019.

De verhoging van het effec

In [71]:
image = cv2.imread("118preprocessed.jpg")
#rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# OCR the image, supplying the country code as the language parameter
config = "--psm 4"
text = pytesseract.image_to_string(image,config=config, lang="fra")

print(text)

Bijlage 2: geldig vanaf 01/02/2021

Annexe 2: valable à partir du 01/02/2021

    

 
  

PER WEEK

PER
PER MAAND PER 3 MAANDEN JAAR

 
 

MEX MAT OR | PARSMORS MONS OR
5,30 17,40 48,50 TR 00
5,80 19,30 54,50 194,00
7,20 6,40 21,30 59,50 212,00
7,90 65,00 231,00
8,50 7,50 24,90 70,00 250,00
9,10 8,00 74,00 266,00
8,50 28,00 79,00 282,00
10,10 8,90 29,70 83,00 298,00
11,30 33,00 92,00 330,00

11,80 10,40 35,00 98,00 348,00

12,40 10,90 36,50 102,00 364,00

13,10 11,50 38,00 107,00 383,00
14,10 12,40 41,50 116,00 415,00
14,80 13,00 43,50 121,00 433,00

L 15,40 13,50 45,00 126,00 449,00



In [108]:
from langdetect import detect


def is_french(stringx) :
    try :
        if detect(stringx.lower()) == 'fr' :
            return True
    except :
            return False

fr_list = []
for line in text_list :
    if is_french(line) :
        fr_list.append(line.replace('\n',' '))
        
    

 

#for line in mixed_text.strip().splitlines():
 # print(line + u"\n")
 # for language in Detector(line).languages:
  #  print(language)
 # print("\n")
 

In [56]:
is_french(text_list[12].lower())

True

In [113]:
fr_list[9]

'Le montant des autres avantages doit étre vérifiable.'

In [93]:
print(text_list[12])

CONVENTION COLLECTIVE DE TRAVAILDU13
NOVEMBRE 2019 CONCLUE AU SEIN DELA
COMMISSION PARITAIRE AUXILIAIRE POUR
OUVRIERS RELATIVE A L’ACCORD SECTORIEL
2019-2020 — POUVOIR D’ACHAT
